In [1]:
import time
import cv2
import numpy as np
import pytesseract

In [11]:
fps = []

In [12]:
net = cv2.dnn.readNet('.\\yolo_data\\yolov3-tiny.weights', '.\\yolo_data\\yolov3-tiny.cfg')
layer_names = net.getLayerNames()
# Get the indices of the unconnected output layers
output_layer_indices = net.getUnconnectedOutLayers()

# Ensure output_layer_indices is a list
if isinstance(output_layer_indices, int):
    output_layer_indices = [output_layer_indices]

# Convert the indices to layer names
output_layers = [layer_names[i - 1] for i in output_layer_indices]

In [13]:
capture = cv2.VideoCapture(0)
#capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
#capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
while capture.isOpened():
    success, image = capture.read()
    res = image.copy()
    height, width, channels = image.shape

    if not success:
      print("Ignoring empty camera frame.")
      continue

    fps.append(time.time())
    if len(fps) > 10:
      fps.pop(0)
      t = fps[9] - fps[0]
      cv2.putText(res, str(int(10/t)), (25,30), cv2.FONT_ITALIC, 0.5, (0,255,0), 2)
    


    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
      for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5:
          center_x = int(detection[0] * width)
          center_y = int(detection[1] * height)

          d_w = int(detection[2] * width)
          d_h = int(detection[3] * height)
          d_x = int(center_x - d_w / 2)
          d_y = int(center_y - d_h / 2)

          boxes.append([d_x, d_y, d_w, d_h])
          confidences.append(float(confidence))
          class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
      if i in indexes:
        r_x, r_y, r_w, r_h = boxes[i]
        cv2.rectangle(res, (r_x, r_y), (r_x + r_w, r_y + r_h), (0,255,0), 2)

    

    ocr_data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    
    for i in range(len(ocr_data['text'])):
      o_x = ocr_data['left'][i]
      o_y = ocr_data['top'][i]
      o_w = ocr_data['width'][i]
      o_h = ocr_data['height'][i]
      
      text = ocr_data['text'][i]
      conf = int(ocr_data['conf'][i])

      if conf > 50 and len(text)>0:
        cv2.rectangle(res, (o_x, o_y), (o_x + o_w, o_y + o_h), (255,0,0), 1)



    cv2.imshow('result', res)


    key_input = cv2.waitKey(1)

    if key_input & 0xFF == 27:
      break

capture.release()
cv2.destroyAllWindows()